In [128]:
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_datetime

In [129]:
# load in the sample data
airnow_df = pd.read_csv("../../Data/Raw/airnow_sample_9_23_25.csv")
aqs_df = pd.read_csv("../../Data/Raw/aqs_sample_9_24_25.csv")
hvo_df = pd.read_csv("../../Data/Raw/hvo_sample_9_24_25.csv")
open_meteo_df = pd.read_csv("../../Data/Raw/openmeteo_sample_9_24_25.csv")
purpleair_df = pd.read_csv("../../Data/Raw/purpleair_sample_9_24_25.csv")

quick loop to print out a basic EDA report for each data frame 

In [130]:
dfs = {
    "airnow": airnow_df,
    "aqs": aqs_df,
    "hvo": hvo_df,
    "openmeteo": open_meteo_df,
    "purpleair": purpleair_df
}
for name, df in dfs.items():
    print(f"{name} size: {df.shape}\n")
    print(f"{name} columns:")
    print(list(df.columns), "\n")
    print(f"{name} datatypes and nulls:")
    print(df.info())
    print("--------------------------------------------")

airnow size: (1, 10)

airnow columns:
['DateObserved', 'HourObserved', 'LocalTimeZone', 'ReportingArea', 'StateCode', 'Latitude', 'Longitude', 'ParameterName', 'AQI', 'Category'] 

airnow datatypes and nulls:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DateObserved   1 non-null      object 
 1   HourObserved   1 non-null      int64  
 2   LocalTimeZone  1 non-null      object 
 3   ReportingArea  1 non-null      object 
 4   StateCode      1 non-null      object 
 5   Latitude       1 non-null      float64
 6   Longitude      1 non-null      float64
 7   ParameterName  1 non-null      object 
 8   AQI            1 non-null      int64  
 9   Category       1 non-null      object 
dtypes: float64(2), int64(2), object(6)
memory usage: 212.0+ bytes
None
--------------------------------------------
aqs size: (6813, 29)

aqs columns:
['state_co

In [131]:
airnow_df.dtypes

DateObserved      object
HourObserved       int64
LocalTimeZone     object
ReportingArea     object
StateCode         object
Latitude         float64
Longitude        float64
ParameterName     object
AQI                int64
Category          object
dtype: object

In [132]:
airnow_df.head()

,DateObserved,HourObserved,LocalTimeZone,ReportingArea,StateCode,Latitude,Longitude,ParameterName,AQI,Category
0,2025-09-24,21,HST,Hilo,HI,19.7175,-155.1103,PM2.5,10,"{'Number': 1, 'Name': 'Good'}"


In [133]:
# Step 1 — AirNow: build UTC timestamps from DateObserved + HourObserved (assumed HST)

# 1) Build a local (HST) timestamp from date + hour
airnow_df["timestamp_hst"] = pd.to_datetime(
    airnow_df["DateObserved"].astype(str) + " " + airnow_df["HourObserved"].astype(str).str.zfill(2) + ":00",
    format="%Y-%m-%d %H:%M",
    errors="coerce"
).dt.tz_localize("Pacific/Honolulu")

# 2) Convert to UTC and align to the hour
airnow_df["timestamp_utc"] = airnow_df["timestamp_hst"].dt.tz_convert("UTC").dt.floor("h")
airnow_df['timestamp_utc'].head()

0   2025-09-25 07:00:00+00:00
Name: timestamp_utc, dtype: datetime64[ns, UTC]

In [134]:
aqs_df.dtypes

state_code                 int64
county_code                int64
site_number                int64
parameter_code             int64
poc                        int64
latitude                 float64
longitude                float64
datum                     object
parameter                 object
date_local                object
time_local                object
date_gmt                  object
time_gmt                  object
sample_measurement       float64
units_of_measure          object
units_of_measure_code      int64
sample_duration           object
sample_duration_code       int64
sample_frequency          object
detection_limit          float64
uncertainty              float64
qualifier                 object
method_type               object
method                    object
method_code                int64
state                     object
county                    object
date_of_last_change       object
cbsa_code                  int64
dtype: object

In [135]:
aqs_df[['date_gmt','date_local','date_of_last_change','time_gmt','time_local']].head(1)

,date_gmt,date_local,date_of_last_change,time_gmt,time_local
0,2022-01-01,2022-01-01,2022-03-29,10:00,00:00


In [136]:
# Step 2 — AQS: build UTC timestamps from date_gmt + time_gmt

# 1) Combine the GMT date and time into a single string and parse to datetime
aqs_df["timestamp_gmt"] = pd.to_datetime(
    aqs_df["date_gmt"].astype(str) + " " + aqs_df["time_gmt"].astype(str),
    format="%Y-%m-%d %H:%M",
    errors="coerce"
)

# 2) These timestamps represent UTC but are tz-naive → localize (assign) UTC, then align to hour
aqs_df["timestamp_utc"] = aqs_df["timestamp_gmt"].dt.tz_localize("UTC").dt.floor("h")

# 3) (Optional) also keep a local HST version for interpretation/reporting
aqs_df["timestamp_hst"] = aqs_df["timestamp_utc"].dt.tz_convert("Pacific/Honolulu")
aqs_df['timestamp_utc'].head()


0   2022-01-01 10:00:00+00:00
1   2022-01-01 11:00:00+00:00
2   2022-01-01 12:00:00+00:00
3   2022-01-01 13:00:00+00:00
4   2022-01-01 14:00:00+00:00
Name: timestamp_utc, dtype: datetime64[ns, UTC]

In [137]:
open_meteo_df.dtypes

date                        object
temperature_2m             float64
relative_humidity_2m       float64
precipitation              float64
rain                       float64
showers                    float64
evapotranspiration         float64
vapour_pressure_deficit    float64
wind_speed_10m             float64
wind_speed_80m             float64
wind_speed_120m            float64
wind_speed_180m            float64
wind_direction_10m         float64
wind_direction_80m         float64
wind_direction_120m        float64
wind_direction_180m        float64
wind_gusts_10m             float64
dtype: object

In [138]:
open_meteo_df['date'].head(1)

0    2025-09-25 00:00:00+00:00
Name: date, dtype: object

In [139]:
open_meteo_df.rename(columns={'date':'timestamp_utc'}, inplace=True)
open_meteo_df.head()

,timestamp_utc,temperature_2m,relative_humidity_2m,precipitation,rain,showers,evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,wind_direction_10m,wind_direction_80m,wind_direction_120m,wind_direction_180m,wind_gusts_10m
0,2025-09-25 00:00:00+00:00,23.906500,66.0,0.0,0.0,0.0,0.20,1.008685,12.889810,16.992609,17.208603,16.923830,54.090195,53.615550,52.650585,51.911175,32.760002
1,2025-09-25 01:00:00+00:00,22.856500,73.0,0.0,0.0,0.0,0.13,0.751881,11.885453,15.696165,15.913465,16.135872,54.865723,53.392834,52.352320,51.340170,30.960001
2,2025-09-25 02:00:00+00:00,22.306500,76.0,0.0,0.0,0.0,0.09,0.646420,10.233123,13.513193,14.003028,14.003028,50.710617,48.239800,46.041542,43.958473,27.720000
3,2025-09-25 03:00:00+00:00,21.856500,77.0,0.0,0.0,0.0,0.06,0.602765,8.669949,12.218805,12.475961,12.496718,48.366554,44.999897,43.830950,41.496380,24.119999
4,2025-09-25 04:00:00+00:00,20.956501,83.0,0.0,0.0,0.0,0.03,0.421652,5.860375,9.178235,9.220499,9.290511,47.489597,41.820090,38.659830,35.537766,20.519999


In [140]:
purpleair_df.dtypes

sensor_index      int64
last_seen         int64
name             object
latitude        float64
longitude       float64
pm2.5_atm       float64
dtype: object

In [141]:
purpleair_df.head()

,sensor_index,last_seen,name,latitude,longitude,pm2.5_atm
0,12062,1758788674,Honokaa,20.067993,-155.46208,0.0
1,70191,1758788663,HAVO Rainshed,19.430813,-155.25780,0.0
2,123663,1758788709,Volcano,19.451317,-155.24246,0.0
3,169685,1758788723,Ninole Makai,19.947739,-155.17891,0.0
4,169961,1758788702,Ainaloa,19.527859,-154.99074,1.2


It took a fair amount of digging to figure out what exactly last_seen is (unix encoding of seconds since a particular date in history) and then how to parse it to match the rest of utc standard encoding.

In [142]:
# Convert last_seen (Unix timestamp) to UTC datetime
purpleair_df["timestamp_utc"] = pd.to_datetime(purpleair_df["last_seen"], unit="s", utc=True)

In [143]:
purpleair_df.head()

,sensor_index,last_seen,name,latitude,longitude,pm2.5_atm,timestamp_utc
0,12062,1758788674,Honokaa,20.067993,-155.46208,0.0,2025-09-25 08:24:34+00:00
1,70191,1758788663,HAVO Rainshed,19.430813,-155.25780,0.0,2025-09-25 08:24:23+00:00
2,123663,1758788709,Volcano,19.451317,-155.24246,0.0,2025-09-25 08:25:09+00:00
3,169685,1758788723,Ninole Makai,19.947739,-155.17891,0.0,2025-09-25 08:25:23+00:00
4,169961,1758788702,Ainaloa,19.527859,-154.99074,1.2,2025-09-25 08:25:02+00:00


In [144]:
purpleair_df.drop(columns='last_seen', inplace=True)
purpleair_df.head()

,sensor_index,name,latitude,longitude,pm2.5_atm,timestamp_utc
0,12062,Honokaa,20.067993,-155.46208,0.0,2025-09-25 08:24:34+00:00
1,70191,HAVO Rainshed,19.430813,-155.25780,0.0,2025-09-25 08:24:23+00:00
2,123663,Volcano,19.451317,-155.24246,0.0,2025-09-25 08:25:09+00:00
3,169685,Ninole Makai,19.947739,-155.17891,0.0,2025-09-25 08:25:23+00:00
4,169961,Ainaloa,19.527859,-154.99074,1.2,2025-09-25 08:25:02+00:00


In [145]:
hvo_df.dtypes

vName                 object
lat                  float64
long                 float64
vnum                   int64
volcanoCd             object
vUrl                  object
vImage                object
obs                   object
region                object
noticeId              object
noticeSynopsis        object
alertLevel            object
colorCode             object
statusIconUrl         object
alertDate             object
colorDate             object
noticeUrl             object
noticeSectionData     object
nvewsThreat           object
dtype: object

In [146]:
hvo_df.head()

,vName,lat,long,vnum,volcanoCd,vUrl,vImage,obs,region,noticeId,noticeSynopsis,alertLevel,colorCode,statusIconUrl,alertDate,colorDate,noticeUrl,noticeSectionData,nvewsThreat
0,Kilauea,19.421,-155.287,332010,hi3,https://www.usgs.gov/volcanoes/kilauea,https://volcanoes.usgs.gov/vsc/images/kilauea/...,hvo,Hawaii,DOI-USGS-HVO-2025-09-24T18:46:03+00:00,HVO Kilauea ORANGE/WATCH - The Halemaʻumaʻu er...,WATCH,ORANGE,https://volcanoes.usgs.gov/images/icons/map/or...,2025-09-24 18:50:02,2025-09-24 18:50:02,https://volcanoes.usgs.gov/hans2/view/notice/D...,https://volcanoes.usgs.gov/vsc/api/hansApi/not...,Very High Threat


In [147]:
hvo_df["timestamp_utc"] = pd.to_datetime(hvo_df["alertDate"], utc=True, errors="coerce").dt.floor('h')
hvo_df['timestamp_utc'].head()

0   2025-09-24 18:00:00+00:00
Name: timestamp_utc, dtype: datetime64[ns, UTC]

In [148]:
import re
# Normalize strings safely
for col in ['alertLevel','colorCode','noticeSynopsis']:
    if col in hvo_df.columns:
        hvo_df[col] = hvo_df[col].astype(str).str.strip()

# Rule sets 
elevated_alert_levels = {'WATCH','WARNING'}
elevated_color_codes  = {'ORANGE','RED'}
elevated_mvews_Threat = {'Very High Threat'}

def text_has_eruption_cues(txt: str) -> bool:
    if not isinstance(txt, str): 
        return False
    # add more cues as needed
    cues = r'(eruption|eruptive|lava|fountain|ash\s+cloud|tephra|effusive)'
    return bool(re.search(cues, txt.lower()))

def make_eruption_flag(row) -> int:
    level = (row.get('alertLevel') or '').upper()
    color = (row.get('colorCode')  or '').upper()
    syn   =  row.get('noticeSynopsis') or ''
    cond_level = level in elevated_alert_levels
    cond_color = color in elevated_color_codes
    cond_text  = text_has_eruption_cues(syn)
    return int(cond_level or cond_color or cond_text)

hvo_df['eruption_flag'] = hvo_df.apply(make_eruption_flag, axis=1).astype('int8')

In [149]:
hvo_df

,vName,lat,long,vnum,volcanoCd,vUrl,vImage,obs,region,noticeId,...,alertLevel,colorCode,statusIconUrl,alertDate,colorDate,noticeUrl,noticeSectionData,nvewsThreat,timestamp_utc,eruption_flag
0,Kilauea,19.421,-155.287,332010,hi3,https://www.usgs.gov/volcanoes/kilauea,https://volcanoes.usgs.gov/vsc/images/kilauea/...,hvo,Hawaii,DOI-USGS-HVO-2025-09-24T18:46:03+00:00,...,WATCH,ORANGE,https://volcanoes.usgs.gov/images/icons/map/or...,2025-09-24 18:50:02,2025-09-24 18:50:02,https://volcanoes.usgs.gov/hans2/view/notice/D...,https://volcanoes.usgs.gov/vsc/api/hansApi/not...,Very High Threat,2025-09-24 18:00:00+00:00,1


In [150]:
purpleair_df.head()

,sensor_index,name,latitude,longitude,pm2.5_atm,timestamp_utc
0,12062,Honokaa,20.067993,-155.46208,0.0,2025-09-25 08:24:34+00:00
1,70191,HAVO Rainshed,19.430813,-155.25780,0.0,2025-09-25 08:24:23+00:00
2,123663,Volcano,19.451317,-155.24246,0.0,2025-09-25 08:25:09+00:00
3,169685,Ninole Makai,19.947739,-155.17891,0.0,2025-09-25 08:25:23+00:00
4,169961,Ainaloa,19.527859,-154.99074,1.2,2025-09-25 08:25:02+00:00


In [151]:
pa = purpleair_df.copy()
pa['timestamp_utc'] = pd.to_datetime(pa['timestamp_utc'], utc=True)
pa['hour'] = pa['timestamp_utc'].dt.floor('h')

# one row per (sensor, hour); keep stable sensor metadata from the first row in the hour
pa_hourly = (
    pa.groupby(['sensor_index','hour'], as_index=False)
      .agg(
          pm25_atm_mean=('pm2.5_atm','mean'),
          name=('name','first'),
          latitude=('latitude','first'),
          longitude=('longitude','first'),
      )
)

# corrections (no humidity available yet)
pa_hourly['pm25_raw_cfatm'] = pa_hourly['pm25_atm_mean']
pa_hourly['pm25_lrapa']     = 0.5 * pa_hourly['pm25_atm_mean'] - 0.66

# cleaned copies: clip to [0, 500]
for col in ['pm25_raw_cfatm','pm25_lrapa']:
    x = pd.to_numeric(pa_hourly[col], errors='coerce')
    pa_hourly[col + '_clean'] = x.clip(lower=0, upper=500)

# tidy index if you like working with a MultiIndex
pa_hourly = pa_hourly.set_index(['hour','sensor_index']).sort_index()


In [152]:
pa_hourly.reset_index(inplace=True)

In [153]:
pa_hourly

,hour,sensor_index,pm25_atm_mean,name,latitude,longitude,pm25_raw_cfatm,pm25_lrapa,pm25_raw_cfatm_clean,pm25_lrapa_clean
0,2025-09-25 08:00:00+00:00,12062,0.0,Honokaa,20.067993,-155.46208,0.0,-0.66,0.0,0.0
1,2025-09-25 08:00:00+00:00,70191,0.0,HAVO Rainshed,19.430813,-155.25780,0.0,-0.66,0.0,0.0
2,2025-09-25 08:00:00+00:00,123663,0.0,Volcano,19.451317,-155.24246,0.0,-0.66,0.0,0.0
3,2025-09-25 08:00:00+00:00,169685,0.0,Ninole Makai,19.947739,-155.17891,0.0,-0.66,0.0,0.0
4,2025-09-25 08:00:00+00:00,169961,1.2,Ainaloa,19.527859,-154.99074,1.2,-0.06,1.2,0.0
5,2025-09-25 08:00:00+00:00,175209,0.0,NEON D20 PUUM,19.553340,-155.31734,0.0,-0.66,0.0,0.0
6,2025-09-25 08:00:00+00:00,192267,0.0,Volcano Village,19.430979,-155.23805,0.0,-0.66,0.0,0.0
7,2025-09-25 08:00:00+00:00,195375,0.1,Namakanipaio Campground,19.425278,-155.29556,0.1,-0.61,0.1,0.0
8,2025-09-25 08:00:00+00:00,210065,0.2,Singing Mountain,19.600403,-154.96260,0.2,-0.56,0.2,0.0


In [ ]:
for name, df in dfs.items():
    print(f"\n{name} — missing % by column")
    missing_pct = df.isna().mean().mul(100).round(2)
    print(missing_pct.sort_values(ascending=False))


airnow — missing % by column
DateObserved     0.0
HourObserved     0.0
LocalTimeZone    0.0
ReportingArea    0.0
StateCode        0.0
Latitude         0.0
Longitude        0.0
ParameterName    0.0
AQI              0.0
Category         0.0
timestamp_hst    0.0
timestamp_utc    0.0
dtype: float64

aqs — missing % by column
uncertainty              100.00
qualifier                 82.71
sample_measurement        12.02
state_code                 0.00
poc                        0.00
latitude                   0.00
site_number                0.00
county_code                0.00
datum                      0.00
parameter                  0.00
time_local                 0.00
date_local                 0.00
date_gmt                   0.00
time_gmt                   0.00
longitude                  0.00
parameter_code             0.00
units_of_measure_code      0.00
units_of_measure           0.00
sample_duration            0.00
sample_duration_code       0.00
detection_limit            0.00
samp

In [160]:
aqs_df.qualifier.unique()

array([nan, 'BA - Maintenance/Routine Repairs.',
       'BR - Sample Value Below Acceptable Range.',
       '1 - Deviation from a CFR/Critical Criteria Requirement.',
       'AN - Machine Malfunction.',
       'AH - Sample Flow Rate or CV out of Limits.',
       'AV - Power Failure.', 'AQ - Collection Error.',
       'AY - Q C Control Points (zero/span).'], dtype=object)

In [161]:
aqs_df.qualifier.value_counts()

qualifier
BA - Maintenance/Routine Repairs.                          581
1 - Deviation from a CFR/Critical Criteria Requirement.    359
AQ - Collection Error.                                     212
AY - Q C Control Points (zero/span).                         9
BR - Sample Value Below Acceptable Range.                    6
AV - Power Failure.                                          4
AH - Sample Flow Rate or CV out of Limits.                   4
AN - Machine Malfunction.                                    3
Name: count, dtype: int64

In [162]:
aqs_df.sample_measurement.nunique()

54

In [163]:
aqs_df.sample_measurement.value_counts()

sample_measurement
 2.0      665
 3.0      630
 4.0      591
 1.0      563
 5.0      552
 6.0      458
 0.0      429
 7.0      356
-1.0      309
 8.0      268
 9.0      203
-2.0      162
 10.0     140
 11.0     114
-3.0       99
 12.0      73
-4.0       73
 13.0      64
-5.0       44
 14.0      39
-6.0       27
 15.0      20
 16.0      19
 17.0      18
-7.0       13
 18.0      11
-8.0        9
-9.0        7
 19.0       5
 20.0       4
 22.0       4
 35.0       2
 23.0       2
 5.7        1
 3.4        1
 2.4        1
 4.2        1
 2.3        1
 26.0       1
 41.0       1
 25.0       1
 27.0       1
 29.0       1
 42.0       1
 34.0       1
 43.0       1
 21.0       1
 58.0       1
 136.0      1
 145.0      1
 91.0       1
 24.0       1
 2.5        1
 6.1        1
Name: count, dtype: int64